In [2]:
!pip install huggingface-hub transformers==4.39.0 scikit-learn pandas sentencepiece protobuf datasets kagglehub

DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.1 -> 25.1
[notice] To update, run: python -m pip install --upgrade pip


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from sklearn import metrics

import torch
from transformers import pipeline
from datasets import Dataset
import os

In [2]:
# Set the Seed for Reproducible Results
seed = 42
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [ ]:
from huggingface_hub import login
# Generate huggingface token
TOKEN = "HUGGINGFACE_TOKEN"
login(token = TOKEN)

In [4]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [5]:
def compute_metrics(y_true, y_pred):
    print("f1_score:", metrics.f1_score(y_true, y_pred, average='weighted'))
    print("precision_score:", metrics.precision_score(y_true, y_pred, average='weighted'))
    print("recall_score:", metrics.recall_score(y_true, y_pred, average='weighted'))

In [6]:
df_train = pd.read_csv(
    "/root/.cache/kagglehub/datasets/abdoukarimkandji/wolbanking77v1/versions/3/wolbanking77v1/5k_split/train/train.csv")

df_test = pd.read_csv(
    "/root/.cache/kagglehub/datasets/abdoukarimkandji/wolbanking77v1/versions/3/wolbanking77v1/5k_split/test/test.csv")

df_train = df_train[['input_wo', 'label']]
df_test = df_test[['input_wo', 'label']]

#removal of punctuation marks
df_train['input_wo_clean'] = df_train['input_wo'].replace(regex=True, to_replace = '[^\\w\\s]', value = ' ')
df_test['input_wo_clean'] = df_test['input_wo'].replace(regex=True, to_replace = '[^\\w\\s]', value = ' ')

# Apply lowercase
df_train['input_wo_clean'] = df_train['input_wo_clean'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df_test['input_wo_clean'] = df_test['input_wo_clean'].apply(lambda x: " ".join(x.lower() for x in x.split()))

X_train = df_train["input_wo_clean"].reset_index(drop=True)
X_test = df_test["input_wo_clean"].reset_index(drop=True)
y_train = df_train["label"].reset_index(drop=True)
y_test = df_test["label"].reset_index(drop=True)

print("Train shape : ", X_train.shape)
print("Test shape : ", X_test.shape)

Train shape :  (4000,)
Test shape :  (1000,)


# bert-base-uncased

In [15]:
classifier = pipeline("zero-shot-classification", model="bert-base-uncased", device=device, batch_size=8, truncation=True)
candidate_labels = list(set(y_test))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


In [16]:
output = []
for x in X_test.to_numpy():
  output.append(classifier(x, candidate_labels, multi_label=False))
# print(output)

--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.10/logging/__init__.py", line 1100, in emit
    msg = self.format(record)
  File "/usr/lib/python3.10/logging/__init__.py", line 943, in format
    return fmt.format(record)
  File "/usr/lib/python3.10/logging/__init__.py", line 678, in format
    record.message = record.getMessage()
  File "/usr/lib/python3.10/logging/__init__.py", line 368, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.10/dist-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 1053, in launch_instance
    app.start(

In [19]:
y_pred = []
for y in output:
  y_pred.append(y['labels'][0])

In [20]:
print(metrics.classification_report(y_test, y_pred))

                                                  precision    recall  f1-score   support

                           Refund not showing up       0.00      0.00      0.00        17
                                activate my card       0.00      0.00      0.00        16
                                       age limit       0.07      0.40      0.12        10
                         apple pay or google pay       0.00      0.00      0.00        14
                                     atm support       0.00      0.00      0.00         9
                                automatic top up       0.00      0.00      0.00        14
         balance not updated after bank transfer       0.00      0.00      0.00        17
balance not updated after cheque or cash deposit       0.00      0.00      0.00        17
                         beneficiary not allowed       0.00      0.00      0.00        15
                                 cancel transfer       0.00      0.00      0.00        14
         

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [22]:
compute_metrics(y_test, y_pred)

f1_score: 0.008317346546272548
precision_score: 0.011685598090843737
recall_score: 0.017


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Davlan/afro-xlmr-large

In [24]:
classifier = pipeline("zero-shot-classification", model="Davlan/afro-xlmr-large", device=device, batch_size=8, truncation=True)
candidate_labels = list(set(y_test))

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at Davlan/afro-xlmr-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


In [25]:
output = []
for x in X_test.to_numpy():
  output.append(classifier(x, candidate_labels, multi_label=False))
# print(output)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
y_pred = []
for y in output:
  y_pred.append(y['labels'][0])
y_pred

In [27]:
print(metrics.classification_report(y_test, y_pred))

                                                  precision    recall  f1-score   support

                           Refund not showing up       0.00      0.00      0.00        17
                                activate my card       0.00      0.00      0.00        16
                                       age limit       0.00      0.00      0.00        10
                         apple pay or google pay       0.00      0.00      0.00        14
                                     atm support       0.00      0.00      0.00         9
                                automatic top up       0.00      0.00      0.00        14
         balance not updated after bank transfer       0.00      0.00      0.00        17
balance not updated after cheque or cash deposit       0.00      0.00      0.00        17
                         beneficiary not allowed       0.00      0.00      0.00        15
                                 cancel transfer       0.00      0.00      0.00        14
         

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [30]:
compute_metrics(y_test, y_pred)

f1_score: 0.01092975156347801
precision_score: 0.018659809527563152
recall_score: 0.012


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# bonadossou/afrolm_active_learning

In [31]:
classifier = pipeline("zero-shot-classification", model="bonadossou/afrolm_active_learning", device=device, batch_size=16, truncation=True)
candidate_labels = list(set(y_test))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/755 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.06G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at bonadossou/afrolm_active_learning and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/241 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/6.01M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


In [32]:
data = {'text': X_test.to_list()}

In [33]:
dataset = Dataset.from_dict(data)
dataset

Dataset({
    features: ['text'],
    num_rows: 1000
})

In [34]:
output = []
for x in dataset:
  output.append(classifier(x['text'], candidate_labels, multi_label=False))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [35]:
y_pred = []
for y in output:
  y_pred.append(y['labels'][0])

In [36]:
print(metrics.classification_report(y_test, y_pred))

                                                  precision    recall  f1-score   support

                           Refund not showing up       0.00      0.00      0.00        17
                                activate my card       0.00      0.00      0.00        16
                                       age limit       0.00      0.00      0.00        10
                         apple pay or google pay       0.00      0.00      0.00        14
                                     atm support       0.01      0.11      0.02         9
                                automatic top up       0.00      0.00      0.00        14
         balance not updated after bank transfer       0.00      0.00      0.00        17
balance not updated after cheque or cash deposit       0.00      0.00      0.00        17
                         beneficiary not allowed       0.00      0.00      0.00        15
                                 cancel transfer       0.00      0.00      0.00        14
         

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [37]:
compute_metrics(y_test, y_pred)

f1_score: 0.002818089576456643
precision_score: 0.001696215782344033
recall_score: 0.009


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [35]:
metrics.f1_score(y_test, y_pred, average='weighted')

0.011347131413624926

# MoritzLaurer/mDeBERTa-v3-base-mnli-xnli

In [13]:
classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli", device=device, batch_size=128, truncation=True)
candidate_labels = list(y_test)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [26]:
data = {'text': X_test.to_list()}
dataset = Dataset.from_dict(data)
dataset

Dataset({
    features: ['text'],
    num_rows: 1000
})

In [27]:
def zero_shot_classifier(classifier, x):
    return classifier(x['text'], candidate_labels, multi_label=False)

In [28]:
# output = []
output = dataset.map(lambda x: zero_shot_classifier(classifier, x))

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [29]:
output[0]['labels'][0]

'age limit'

In [30]:
y_pred = []
for y in output:
  y_pred.append(y['labels'][0])

In [31]:
print(metrics.classification_report(y_test, y_pred))

                                                  precision    recall  f1-score   support

                           Refund not showing up       0.00      0.00      0.00        17
                                activate my card       1.00      0.06      0.12        16
                                       age limit       0.01      0.20      0.02        10
                         apple pay or google pay       0.00      0.00      0.00        14
                                     atm support       0.00      0.00      0.00         9
                                automatic top up       0.80      0.29      0.42        14
         balance not updated after bank transfer       0.00      0.00      0.00        17
balance not updated after cheque or cash deposit       0.00      0.00      0.00        17
                         beneficiary not allowed       0.14      0.07      0.09        15
                                 cancel transfer       0.00      0.00      0.00        14
         

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [32]:
metrics.f1_score(y_test, y_pred, average='weighted')

0.026400031345717907

In [33]:
compute_metrics(y_test, y_pred)

f1_score: 0.026400031345717907
precision_score: 0.06552751598176082
recall_score: 0.028


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# castorini/afriteva_v2_base

In [34]:
classifier = pipeline("zero-shot-classification", model="castorini/afriteva_v2_base", device=device, batch_size=8, truncation=True)
candidate_labels = list(set(y_test))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.72G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at castorini/afriteva_v2_base and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/2.31k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/2.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Failed to determine 'entailment' label i

In [35]:
data = {'text': X_test.to_list()}
dataset = Dataset.from_dict(data)
dataset

Dataset({
    features: ['text'],
    num_rows: 1000
})

In [36]:
def zero_shot_classifier(classifier, x):
    return classifier(x['text'], candidate_labels, multi_label=False)

In [37]:
output = dataset.map(lambda x: zero_shot_classifier(classifier, x))

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [38]:
y_pred = []
for y in output:
  y_pred.append(y['labels'][0])

In [39]:
print(metrics.classification_report(y_test, y_pred))

                                                  precision    recall  f1-score   support

                           Refund not showing up       0.00      0.00      0.00        17
                                activate my card       0.00      0.00      0.00        16
                                       age limit       0.00      0.00      0.00        10
                         apple pay or google pay       0.03      0.07      0.04        14
                                     atm support       0.00      0.00      0.00         9
                                automatic top up       0.00      0.00      0.00        14
         balance not updated after bank transfer       0.00      0.00      0.00        17
balance not updated after cheque or cash deposit       0.00      0.00      0.00        17
                         beneficiary not allowed       0.00      0.00      0.00        15
                                 cancel transfer       0.01      0.21      0.02        14
         

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [40]:
compute_metrics(y_test, y_pred)

f1_score: 0.007566901600341497
precision_score: 0.007360111245354419
recall_score: 0.013


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
